In [1]:
# Step 1 - import packages and get environment variables

from sqlalchemy import create_engine
import dotenv
from faker import Faker

import os
import pandas as pd
from sqlalchemy import create_engine
import warnings

# Instance Faker
fake = Faker()

# Load .env
dotenv.load_dotenv()

DB_PASS = os.getenv('DB_PASS')

In [2]:
# Step 2 - jupyter notebook configuration

# Warnings ignore
warnings.simplefilter('ignore')

# Show complete dataset in
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib
%matplotlib inline

In [3]:
# Step 3 - import csvs
actor=pd.read_csv('./data/actor.csv')
category=pd.read_csv('./data/category.csv')
film=pd.read_csv('./data/film.csv')
inventory=pd.read_csv('./data/inventory.csv')
language=pd.read_csv('./data/language.csv')
rental=pd.read_csv('./data/rental.csv')
film_actor=pd.read_csv('./data/old_HDD.csv')

In [4]:
# Step 4 - Check if dataframes have null values
nan_cols_actor=actor.isna().sum()
print("\nACTOR NULL COLUMNS:")
print(nan_cols_actor[nan_cols_actor>0])

nan_cols_category=category.isna().sum()
print("\nCATEGORY NULL COLUMNS:")
print(nan_cols_category[nan_cols_category>0])

nan_cols_film=film.isna().sum()
print("\nFILM NULL COLUMNS:")
print(nan_cols_film[nan_cols_film>0])

nan_cols_inventory=inventory.isna().sum()
print("\nINVENTORY NULL COLUMNS:")
print(nan_cols_inventory[nan_cols_inventory>0])

nan_cols_language=language.isna().sum()
print("\nLANGUAGE NULL COLUMNS:")
print(nan_cols_language[nan_cols_language>0])

nan_cols_rental=rental.isna().sum()
print("\nRENTAL NULL COLUMNS")
print(nan_cols_rental[nan_cols_rental>0])

nan_cols_film_actor=film_actor.isna().sum()
print("\nfilm_actor NULL COLUMNS")
print(nan_cols_film_actor[nan_cols_film_actor>0])


ACTOR NULL COLUMNS:
Series([], dtype: int64)

CATEGORY NULL COLUMNS:
Series([], dtype: int64)

FILM NULL COLUMNS:
original_language_id    1000
dtype: int64

INVENTORY NULL COLUMNS:
Series([], dtype: int64)

LANGUAGE NULL COLUMNS:
Series([], dtype: int64)

RENTAL NULL COLUMNS
Series([], dtype: int64)

film_actor NULL COLUMNS
Series([], dtype: int64)


In [5]:
# Step 5 - Delete original_language_id col and all "last_update" columns
film.drop("original_language_id", axis=1, inplace=True)

actor.drop("last_update", axis=1, inplace=True)
category.drop("last_update", axis=1, inplace=True)
film.drop("last_update", axis=1, inplace=True)
inventory.drop("last_update", axis=1, inplace=True)
language.drop("last_update", axis=1, inplace=True)
rental.drop("last_update", axis=1, inplace=True)

In [6]:
# Step 6 - Check if there are duplicated rows
print(actor.duplicated().sum())
print(category.duplicated().sum())
print(film.duplicated().sum())
print(inventory.duplicated().sum())
print(language.duplicated().sum())
print(rental.duplicated().sum())
print(film_actor.duplicated().sum())

0
0
0
0
0
0
0


In [7]:
# Step 7 - Check if there are duplicated rows in some specific subsets
# In order to make the relations with old_HDD.csv data

# 7.1 - It's necessary to check in the film's dataframe that all films titles are unique
print(film.duplicated(subset=['title']).sum())

# 7.2 - It's necessary to check in actor's dataframe that the tupla (first_name-last_name) are unique
print(actor.duplicated(subset=['first_name', 'last_name']).sum())

0
1


In [8]:
# Step 8 - There is one duplicated in the tupla (first_name-last_name) of actor's dataframe.
# We need to delete it, in order to not have ambiguous relations.
actor.drop_duplicates(subset =["first_name", "last_name"], inplace = True)

print(actor.duplicated(subset=['first_name', 'last_name']).sum())

0


In [9]:
# Step 9 - Check if there are more than one category from "film" dataframe
print(film_actor.groupby('title')['category_id'].transform('nunique').max())

1


In [10]:
# Step 10 - Check if there is correspondence one to one
correspondence = rental[["inventory_id", "rental_id"]].set_index('inventory_id').join(inventory.set_index('inventory_id'), how='left')
print(correspondence.sample(10).sort_values(by=['inventory_id']))

              rental_id  film_id  store_id
inventory_id                              
712                 825    155.0       2.0
1686                398      NaN       NaN
1921                963      NaN       NaN
2817                896      NaN       NaN
3132                805      NaN       NaN
3193                156      NaN       NaN
3250                760      NaN       NaN
3664                713      NaN       NaN
4433                999      NaN       NaN
4443                 11      NaN       NaN


In [11]:
# Step 11 - There is only correspondence up to "inventory_id" 1000
# For that reason, we need to transform the data of "inventory_id" col in "rental" dataframe
# We follow Quique's strategy: apply a lambda that return the module (x % 1000)
rental.inventory_id=rental.inventory_id.apply(lambda x : x % 1000)

In [12]:
# Step 12 - Check max value of inventory_id on "inventory" dataframe
print(inventory.inventory_id.max())

1000


In [13]:
# Step 13 - There is not more than one category from film.
# For that reason, we could join film dataframe with film_actor dataframe by title...
film = film.set_index('title').join(film_actor[["title", "category_id"]].set_index('title'), how='left')

In [14]:
# Step 14 - ...and then drop duplicates
film.drop_duplicates(inplace = True)

In [15]:
# Step 15 - There are some films with category and for this reason we created an "Uncategorized" category
new = pd.DataFrame({'category_id': [17], 'name' : ['Uncategorized']})
category = pd.concat([category, new], ignore_index = True, axis = 0)

In [16]:
# Set 16 - Fill all NaN "category_id" fields with the "Uncategorized" id created in the last step
film.category_id.fillna(17, inplace=True)

In [17]:
# Step 17 - Reset index
film.reset_index(inplace=True)

In [18]:
# Step 18 - Transform "category_id" type to integer
film.category_id = film.category_id.astype('int')

In [19]:
# Step 19 - Join
film_actor = film_actor.set_index('title').join(film[["film_id", "title"]].set_index('title'), how='left')

In [20]:
# Step 20 - Reset index
film_actor.reset_index(inplace=True)

In [21]:
# Step 21 - Join
film_actor = film_actor[["first_name", "last_name", "film_id"]].set_index(["first_name", "last_name"]).join(actor[["actor_id", "first_name", "last_name"]].set_index(["first_name", "last_name"]), how='left')

In [22]:
# Step 22 - Reset index
film_actor.reset_index(drop=True, inplace=True)

In [23]:
# Step 23 - Create MySQL engine
str_conn='mysql+pymysql://root:' + DB_PASS + '@localhost:3306'
motor=create_engine(str_conn)

In [24]:
# Step 24 - Create database
delete_db = 'DROP DATABASE IF EXISTS videostore;'
motor.execute(delete_db)
create_db = 'CREATE DATABASE IF NOT EXISTS videostore;'
motor.execute(create_db)

In [25]:
# Step 25 - Use database
use_db = 'use videostore;'
motor.execute(use_db)
# db=conn.connect(host='localhost', user='root', passwd='password', database='productos')
# cursor=db.cursor()

In [26]:
# Step 26 - Create "category" table
create_category_table = '''
    CREATE TABLE IF NOT EXISTS category (
        category_id int PRIMARY KEY,
        name VARCHAR(100)
    );
'''
motor.execute(create_category_table)

In [27]:
# Step 27 - Seed "category" table from "category" dataframe
for i in range(len(category)):
    insert_query=f"INSERT INTO category ({','.join(category.columns)}) VALUES {tuple(category.iloc[i].values)};"
    motor.execute(insert_query)

In [28]:
# Step 28 - Create "actor" table
create_language_table = '''
    CREATE TABLE IF NOT EXISTS language (
        language_id INT PRIMARY KEY,
        name VARCHAR(100)
    );
'''
motor.execute(create_language_table)

In [29]:
# Step 29 - Seed "language" table from "language" dataframe
for i in range(len(language)):
    insert_query=f"INSERT INTO language ({','.join(language.columns)}) VALUES {tuple(language.iloc[i].values)};"
    motor.execute(insert_query)

In [30]:
# Step 30 - Create "film" table
create_film_table = '''
    CREATE TABLE IF NOT EXISTS film (
        title VARCHAR(100),
        film_id INT PRIMARY KEY,
        description VARCHAR(200),
        release_year INT,
        language_id INT,
        rental_duration INT,
        rental_rate FLOAT,
        length INT,
        replacement_cost FLOAT,
        rating VARCHAR(50),
        special_features VARCHAR(100),
        category_id INT,
        FOREIGN KEY (language_id) REFERENCES language(language_id),
        FOREIGN KEY (category_id) REFERENCES category(category_id)
    );
'''
motor.execute(create_film_table)

In [31]:
# Step 31 - Seed "film" table from "film" dataframe
for i in range(len(film)):
    insert_query=f"INSERT INTO film ({','.join(film.columns)}) VALUES {tuple(film.iloc[i].values)};"
    motor.execute(insert_query)

In [32]:
# Step 32 - Create "actor" table
create_actor_table = '''
    CREATE TABLE IF NOT EXISTS actor (
        actor_id INT PRIMARY KEY,
        first_name VARCHAR(100),
        last_name VARCHAR(100)
    );
'''
motor.execute(create_actor_table)

In [33]:
# Step 33 - Seed "actor" table from "actor" dataframe
for i in range(len(actor)):
    insert_query=f"INSERT INTO actor ({','.join(actor.columns)}) VALUES {tuple(actor.iloc[i].values)};"
    motor.execute(insert_query)

In [34]:
# Step 34 - Create "film_actor" table
create_film_actor_table = '''
    CREATE TABLE IF NOT EXISTS film_actor (
        film_id INT,
        actor_id INT,
        PRIMARY KEY (film_id, actor_id),
        FOREIGN KEY (film_id) REFERENCES film(film_id),
        FOREIGN KEY (actor_id) REFERENCES actor(actor_id)
    );
'''
motor.execute(create_film_actor_table)

In [35]:
# Step 35 - BONUS: Create "customer" table
create_customer_table = '''
    CREATE TABLE IF NOT EXISTS customer (
        customer_id INT PRIMARY KEY,
        first_name VARCHAR(100),
        last_name VARCHAR(100),
        address VARCHAR(100),
        phone VARCHAR(50)
    );
'''
motor.execute(create_customer_table)

In [36]:
# Step 36 - BONUS: Seed "customer" table from "faker" library
for i in range(1,601):
    insert_query=f"INSERT INTO customer (customer_id, first_name, last_name, address, phone) VALUES {i, fake.first_name(), fake.last_name(), fake.address(), fake.phone_number() };"
    motor.execute(insert_query)

In [37]:
# Step 37 - BONUS: Create "staff" table
create_staff_table = '''
    CREATE TABLE IF NOT EXISTS staff (
        staff_id INT PRIMARY KEY,
        first_name VARCHAR(100),
        last_name VARCHAR(100),
        address VARCHAR(100),
        phone VARCHAR(50)
    );
'''
motor.execute(create_staff_table)

In [38]:
# Step 38 - BONUS: Seed "staff" table from "faker" dataframe
for i in range(1,3):
    insert_query=f"INSERT INTO staff (staff_id, first_name, last_name, address, phone) VALUES {i, fake.first_name(), fake.last_name(), fake.address(), fake.phone_number() };"
    motor.execute(insert_query)

In [39]:
# Step 39 - BONUS: Create "store" table
create_store_table = '''
    CREATE TABLE IF NOT EXISTS store (
        store_id INT PRIMARY KEY,
        address VARCHAR(100),
        phone VARCHAR(50),
        cif VARCHAR(50)
    );
'''
motor.execute(create_store_table)

In [40]:
# Step 40 - BONUS: Seed "store" table from "faker" dataframe
for i in range(1,3):
    insert_query=f"INSERT INTO store (store_id, address, phone, cif) VALUES {i, fake.address(), fake.phone_number(), fake.ssn() };"
    motor.execute(insert_query)

In [41]:
# Step 41 - Seed "film_actor" table from "film_actor" dataframe
for i in range(len(film_actor)):
    insert_query=f"INSERT INTO film_actor ({','.join(film_actor.columns)}) VALUES {tuple(film_actor.iloc[i].values)};"
    motor.execute(insert_query)

In [42]:
# Step 42 - Create "inventory" table
create_inventory_table = '''
    CREATE TABLE IF NOT EXISTS inventory (
        inventory_id INT PRIMARY KEY,
        film_id INT,
        store_id INT,
        FOREIGN KEY (film_id) REFERENCES film(film_id),
        FOREIGN KEY (store_id) REFERENCES store(store_id)
    );
'''
motor.execute(create_inventory_table)

In [43]:
# Step 43 - Seed "inventory" table from "inventory" dataframe
for i in range(len(inventory)):
    insert_query=f"INSERT INTO inventory ({','.join(inventory.columns)}) VALUES {tuple(inventory.iloc[i].values)};"
    motor.execute(insert_query)

In [44]:
# Step 44 - Create "rental" table
create_rental_table = '''
    CREATE TABLE IF NOT EXISTS rental (
        rental_id INT PRIMARY KEY,
        rental_date DATETIME,
        inventory_id INT,
        customer_id INT,
        return_date DATETIME,
        staff_id INT,
        FOREIGN KEY (inventory_id) REFERENCES inventory(inventory_id),
        FOREIGN KEY (customer_id) REFERENCES customer(customer_id)
    );
'''
motor.execute(create_rental_table)

In [45]:
# Step 45 - Seed "rental" table from "rental" dataframe
for i in range(len(rental)):
    insert_query=f"INSERT INTO rental ({','.join(rental.columns)}) VALUES {tuple(rental.iloc[i].values)};"
    motor.execute(insert_query)

In [46]:
## Step 46 - TOP 5 ACTORS WITH MORE FILMS
top_5_actors='''
SELECT CONCAT(a.first_name, ' ', a.last_name) AS ACTOR, count(f.film_id) AS NUM_FILMS FROM actor AS a
LEFT JOIN film_actor AS fa on a.actor_id = fa.actor_id
LEFT JOIN film AS f on fa.film_id = f.film_id
GROUP BY a.actor_id
ORDER BY NUM_FILMS DESC
LIMIT 5
'''

print(list(motor.execute(top_5_actors)))

[('SANDRA KILMER', 37), ('VAL BOLGER', 35), ('UMA WOOD', 35), ('JULIA MCQUEEN', 33), ('RIP CRAWFORD', 33)]


In [47]:
## Step 47 - TOP 5 FILMS BY RENTAL
top_5_films='''
SELECT f.title AS FILM, count(r.rental_id) AS NUM_RENTAL
FROM rental AS r
LEFT JOIN inventory i on i.inventory_id = r.inventory_id
LEFT JOIN film f on i.film_id = f.film_id
GROUP BY f.film_id
ORDER BY NUM_RENTAL DESC
LIMIT 5
'''

print(list(motor.execute(top_5_films)))

[('BOOGIE AMELIE', 16), ('ATLANTIS CAUSE', 12), ('AFFAIR PREJUDICE', 12), ('BINGO TALENTED', 11), ('BLADE POLISH', 10)]


In [48]:
## Step 48 - TOP 5 CATEGORIES
top_5_categories='''
SELECT c.name AS CATEGORY, count(f.film_id) AS NUM_FILMS
FROM category AS c
LEFT JOIN film f on c.category_id = f.category_id
GROUP BY c.category_id
ORDER BY NUM_FILMS DESC
LIMIT 5
'''

print(list(motor.execute(top_5_categories)))

[('Uncategorized', 386), ('Animation', 50), ('Sports', 46), ('New', 45), ('Horror', 45)]


In [49]:
## Step 49 - TOP 5 LONGEST MOVIES
top_5_longest_movies='''
SELECT f.title AS FILM, f.length AS DURATION_IN_MINUTES
FROM film AS f
ORDER BY f.length DESC
LIMIT 5
'''

print(list(motor.execute(top_5_longest_movies)))

[('GANGS PRIDE', 185), ('HOME PITY', 185), ('CHICAGO NORTH', 185), ('CONTROL ANTHEM', 185), ('DARN FORRESTER', 185)]


In [50]:
## Step 50 - TOP 5 FILMS BY RENTAL INCOMING
top_5_films_by_rental_incoming='''
SELECT f.title AS FILM, sum(f.rental_rate) AS FILM_INCOMING
FROM rental AS r
LEFT JOIN inventory i on i.inventory_id = r.inventory_id
LEFT JOIN film f on i.film_id = f.film_id
GROUP BY f.film_id
ORDER BY FILM_INCOMING DESC
LIMIT 5
'''

print(list(motor.execute(top_5_films_by_rental_incoming)))

[('BOOGIE AMELIE', 79.83999633789062), ('BORN SPINAL', 49.89999771118164), ('BUCKET BROTHERHOOD', 49.89999771118164), ('BROOKLYN DESERT', 44.90999794006348), ('CANDLES GRAPES', 44.90999794006348)]


In [51]:
## Step 51 - TOP 5 CUSTOMERS BY RENTAL FILMS
top_5_customers_by_rental_films='''
SELECT CONCAT(c.first_name, ' ', c.last_name) AS CUSTOMER, count(r.rental_id) AS RENTAL_FILMS
FROM rental AS r
LEFT JOIN customer AS c on r.customer_id = c.customer_id
GROUP BY c.customer_id
ORDER BY RENTAL_FILMS DESC
LIMIT 5
'''

print(list(motor.execute(top_5_customers_by_rental_films)))

[('Lisa Alvarez', 8), ('Troy Perez', 6), ('Rachel Finley', 6), ('Cynthia Gibbs', 6), ('Alexander Castro', 6)]


In [52]:
## Step 52 - TOP 2 STAFF BY RENTAL FILMS
top_2_staff_by_rental_films='''
SELECT CONCAT(s.first_name, ' ', s.last_name) AS STAFF, count(r.rental_id) AS RENTAL_FILMS
FROM rental AS r
LEFT JOIN staff AS s ON r.staff_id = s.staff_id
GROUP BY s.staff_id
ORDER BY RENTAL_FILMS DESC
LIMIT 2
'''

print(list(motor.execute(top_2_staff_by_rental_films)))

[('Kara Mclean', 521), ('Ann Nunez', 479)]


In [53]:
## Step 53 - TOP 2 STORES BY RENTAL FILMS
top_2_stores_by_rental_films='''
SELECT CONCAT('STORE CIF:', s.cif) AS STORE, count(r.rental_id) AS STORE
FROM rental AS r
LEFT JOIN inventory AS i ON r.inventory_id = i.inventory_id
LEFT JOIN store AS s ON s.store_id = i.store_id
GROUP BY s.store_id
ORDER BY STORE DESC
LIMIT 2
'''

print(list(motor.execute(top_2_stores_by_rental_films)))

[('STORE CIF:758-65-1805', 511), ('STORE CIF:104-56-0566', 489)]
